In [3]:
!pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install pytesseract
# https://codetoprosper.com/tesseract-ocr-for-windows

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
import os
import fitz
import pytesseract
import pandas as pd
from PIL import Image
import io
import cv2
import numpy as np

In [53]:
def extract_images_and_text(pdf_path, output_folder, text_window=100):
    # Create an empty list to store image and text information
    image_text_info_list = []

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    

    # Set up pytesseract
    pytesseract.pytesseract.tesseract_cmd = r'C:\Tesseract-OCR\tesseract.exe'  # Change to your Tesseract executable path

    for page_number, page in enumerate(pdf_document):
        # Get all the images on the current page
        images = page.get_images(full=True)
        on_of_images = len(images)
        
        if (on_of_images <= 10) :
            for img_index, img in enumerate(images):
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            image_data = base_image["image"]

            # Determine the file extension based on the image type
            if base_image["ext"] == ".jpg":
                image_ext = "jpg"
            elif base_image["ext"] == ".png":
                image_ext = "png"
            else:
                image_ext = "png"

            # Create a dictionary with image information
            image_info = {
                "File Name": os.path.basename(pdf_path),
                "Image Name": f"page_{page_number + 1}_image_{img_index + 1}.{image_ext}",
                "Page Number": page_number + 1
            }

            # Extract text content before and after the image
            text_before = page.get_text("text", clip=(0, 0, img[2], img[3]-text_window))
            text_after = page.get_text("text", clip=(0, img[3], img[2], img[3]+text_window))

            # Convert image data to a format that Tesseract supports
            image_pil = Image.open(io.BytesIO(image_data))
            image_data = image_pil.convert("RGB")

            # Use OCR to extract text inside the image
            image_text = pytesseract.image_to_string(image_data)

            image_info["Text Before"] = text_before.strip()
            image_info["Text After"] = text_after.strip()
            image_info["Text Inside Image"] = image_text.strip()
            

            # Create the output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)

            # Save the image to the output folder
            image_filename = os.path.join(output_folder, image_info["Image Name"])
            image_pil.save(image_filename, format=image_ext)
            #with open(image_filename, "wb") as image_file:
            #    image_file.write(image_pil.tobytes())
            
            # Append the image and text information to the list
            image_text_info_list.append(image_info)
        

    pdf_document.close()

    # Create a Pandas DataFrame from the image and text information list
    df = pd.DataFrame(image_text_info_list)
    return df

In [54]:
pdf_files = os.listdir("C:/Thesis/CSR Reports/")
print(pdf_files)

['3M_2016.pdf', 'AaltoUniversity_2015.pdf']


In [56]:
# Reading all the files and extract images 
count = 0

damage_list = []
for pdf in pdf_files:
    count = count+1
    #pdffile = "CSR Reports/"+pdf
    
    pdf_file = "C:/Thesis/CSR Reports/" + pdf  # PDF file path
    output_dir = "C:/Thesis/Images/" + pdf[:-4]  #
    #output_dir = "1time_2011"  # Replace with the output folder path
    
    try:
        df = extract_images_and_text(pdf_file, output_dir)
        if count == 1:
            tatal_df = df
        else:
            tatal_df = tatal_df.append(df,ignore_index=True) 
      
    except:
        damage_list.append(pdf)
        

In [48]:
print(tatal_df)

          File Name             Image Name  Page Number  \
0    1time_2011.pdf     page_2_image_1.png            2   
1    1time_2011.pdf     page_2_image_2.png            2   
2    1time_2011.pdf     page_3_image_1.png            3   
3    1time_2011.pdf     page_3_image_2.png            3   
4    1time_2011.pdf     page_3_image_3.png            3   
..              ...                    ...          ...   
164     3M_2016.pdf  page_171_image_10.png          171   
165     3M_2016.pdf  page_171_image_11.png          171   
166     3M_2016.pdf  page_171_image_12.png          171   
167     3M_2016.pdf  page_171_image_13.png          171   
168     3M_2016.pdf  page_171_image_14.png          171   

                                           Text Before  \
0    | highlights 1 | 5-year review 2 | directorate...   
1    | highlights 1 | 5-year review 2 | directorate...   
2                                                        
3                                                        
4

In [8]:
#df = extract_images_and_text(pdf_file, output_dir)

# Print the DataFrame (optional)


         File Name           Image Name  Page Number  \
0   1time_2011.pdf   page_2_image_1.png            2   
1   1time_2011.pdf   page_2_image_2.png            2   
2   1time_2011.pdf   page_3_image_1.png            3   
3   1time_2011.pdf   page_3_image_2.png            3   
4   1time_2011.pdf   page_3_image_3.png            3   
5   1time_2011.pdf   page_3_image_4.png            3   
6   1time_2011.pdf   page_3_image_5.png            3   
7   1time_2011.pdf   page_3_image_6.png            3   
8   1time_2011.pdf   page_3_image_7.png            3   
9   1time_2011.pdf   page_3_image_8.png            3   
10  1time_2011.pdf   page_3_image_9.png            3   
11  1time_2011.pdf  page_3_image_10.png            3   
12  1time_2011.pdf  page_3_image_11.png            3   
13  1time_2011.pdf  page_3_image_12.png            3   
14  1time_2011.pdf  page_3_image_13.png            3   
15  1time_2011.pdf  page_3_image_14.png            3   
16  1time_2011.pdf  page_3_image_15.png         

In [28]:
tatal_df.columns

Index(['File Name', 'Image Name', 'Page Number', 'Text Before', 'Text After',
       'Text Inside Image'],
      dtype='object')

In [43]:
tatal_df.head()

,File Name,Image Name,Page Number,Text Before,Text After,Text Inside Image
0,1time_2011.pdf,page_2_image_1.png,2,| highlights 1 | 5-year review 2 | directorate...,,
1,1time_2011.pdf,page_2_image_2.png,2,| highlights 1 | 5-year review 2 | directorate...,,
2,1time_2011.pdf,page_3_image_1.png,3,,,
3,1time_2011.pdf,page_3_image_2.png,3,,,
4,1time_2011.pdf,page_3_image_3.png,3,,,


In [49]:
tatal_df.to_csv('SaveFiles/ExtractedData.csv', index=False)